In [16]:
from pmdarima import auto_arima
from sklearn.metrics import mean_squared_error,mean_absolute_error
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import MinMaxScaler, StandardScaler

In [17]:
# 創建時間窗口的訓練集數據
def create_dataset(closing_prices, time_window):
    features,targets = [],[]
    for i in range(len(closing_prices) - time_window):
        features.append(closing_prices.iloc[i:i+time_window])
        targets.append(closing_prices.iloc[i+time_window])   
    return np.array(features),np.array(targets)

In [18]:
# load dataset
file_list=['BTCUSD-1m1h','BTCUSD-all','BTCUSD-N2Y','BTCUSD-N4Y',
           'ETHUSD-1m1h','ETHUSD-all','ETHUSD-N2Y','ETHUSD-N4Y',           
           'USDTUSD-1m1h','USDTUSD-all','USDTUSD-N2Y','USDTUSD-N4Y',  
           'BNBUSD-1m1h','BNBUSD-all','BNBUSD-N2Y','BNBUSD-N4Y']


spilt_list=[0.1]
look_backs=[1,7,12,14,24]


files_list=list()
spilts_list=list()
Classifiers_list=list()
lb_list=list()

train_mae=list()
train_rmse=list()
test_mae=list()
test_rmse=list()


In [19]:
for file in file_list:
    for splits in spilt_list:
        for lb in look_backs:
            files_list.append(file)
            spilts_list.append(splits)
            lb_list.append(lb)

            data = pd.read_csv(f'../Data/{file}.csv') 
            train_data, test_data = train_test_split(data['Close'], test_size=splits, shuffle=False)

        
            trainX, trainY = create_dataset(train_data, lb)
            testX, testY = create_dataset(test_data, lb)
            tscv = TimeSeriesSplit(n_splits=5)
            best_model = None
            best_rmse = np.inf

            # for train_index, val_index in tscv.split(trainY):
            #     train_val_data, val_data = trainY[train_index], trainY[val_index]    
            #     model = auto_arima(train_val_data,exogenous=trainX[train_index],max_p=20,max_q=20,seasonal=False)   
            #     forecast = model.predict(len(val_data))
            #     rmse = np.sqrt(mean_squared_error(val_data, forecast))
            #     if rmse < best_rmse:
            #         best_rmse = rmse
            #         best_model = model
            # best_model.fit(trainY)
            
            best_model = auto_arima(trainY,exogenous=trainX,max_p=20,max_q=20,seasonal=False)
            print(f'資料集{file} {splits} {lb}')
            best_model_params = best_model.get_params()['order']
            print("最佳模型參數:ARIMA", best_model_params)
            Classifiers_list.append(f'ARIMA{best_model_params}')
            
            # forecast
            forecast = best_model.predict(n_periods=len(testY))
                

            test_data_mae=mean_absolute_error(testY,forecast)
            test_data_rmse=mean_squared_error(testY,forecast)**0.5        
            test_mae.append(test_data_mae)
            test_rmse.append(test_data_rmse)       

            # predict train
            train_forecast = best_model.predict_in_sample()    

            train_data_mae=mean_absolute_error(trainY,train_forecast)
            train_data_rmse=mean_squared_error(trainY,train_forecast)**0.5
            train_mae.append(train_data_mae)
            train_rmse.append(train_data_rmse) 

            print(f'MAE train:{train_data_mae:.4f} test:{test_data_mae:.4f}')   
            print(f'RMSE train:{train_data_rmse:.4f} test:{test_data_rmse:.4f}')         
        
        

資料集BTCUSD-1m1h 0.1 1
最佳模型參數:ARIMA (0, 1, 1)
MAE train:155.6788 test:548.2686
RMSE train:1365.7935 test:609.0861
資料集BTCUSD-1m1h 0.1 7
最佳模型參數:ARIMA (0, 1, 0)
MAE train:157.4716 test:574.7002
RMSE train:1370.3773 test:622.6985
資料集BTCUSD-1m1h 0.1 12
最佳模型參數:ARIMA (0, 1, 1)
MAE train:157.4548 test:624.2465
RMSE train:1389.1285 test:659.8159
資料集BTCUSD-1m1h 0.1 14
最佳模型參數:ARIMA (2, 1, 2)
MAE train:156.1898 test:624.0290
RMSE train:1367.9660 test:653.5557
資料集BTCUSD-1m1h 0.1 24
最佳模型參數:ARIMA (0, 1, 0)
MAE train:158.8388 test:620.2826
RMSE train:1427.2904 test:644.2350
資料集BTCUSD-all 0.1 1
最佳模型參數:ARIMA (2, 1, 2)
MAE train:358.7688 test:10327.1901
RMSE train:791.4866 test:11115.0236
資料集BTCUSD-all 0.1 7
最佳模型參數:ARIMA (2, 1, 2)
MAE train:359.4574 test:10526.8136
RMSE train:792.2723 test:11227.8257
資料集BTCUSD-all 0.1 12
最佳模型參數:ARIMA (2, 1, 2)
MAE train:359.9967 test:10694.6495
RMSE train:792.9209 test:11325.3603
資料集BTCUSD-all 0.1 14
最佳模型參數:ARIMA (3, 1, 2)
MAE train:358.1614 test:11406.8742
RMSE train:791.

In [20]:
Summary={'Data':files_list,'Classifier':Classifiers_list,'Test Split':spilts_list,'look back':lb_list,
            'train_mae':train_mae,          
            'train_rmse':train_rmse,          
            'test_mae':test_mae,           
            'test_rmse':test_rmse            
         }

df_Summary = pd.DataFrame(Summary)
df_Summary.to_excel("Summary-AutoARIMA(windowT0.1).xlsx",index=False) 
df_Summary

,Data,Classifier,Test Split,look back,train_mae,train_rmse,test_mae,test_rmse
0,BTCUSD-1m1h,"ARIMA(0, 1, 1)",0.1,1,155.678771,1365.793473,548.268631,609.086054
1,BTCUSD-1m1h,"ARIMA(0, 1, 0)",0.1,7,157.471632,1370.377327,574.700240,622.698462
2,BTCUSD-1m1h,"ARIMA(0, 1, 1)",0.1,12,157.454808,1389.128491,624.246502,659.815859
3,BTCUSD-1m1h,"ARIMA(2, 1, 2)",0.1,14,156.189765,1367.965957,624.028988,653.555708
4,BTCUSD-1m1h,"ARIMA(0, 1, 0)",0.1,24,158.838823,1427.290441,620.282633,644.235028
...,...,...,...,...,...,...,...,...
75,BNBUSD-N4Y,"ARIMA(2, 1, 2)",0.1,1,8.605536,26.326215,13.290258,18.097188
76,BNBUSD-N4Y,"ARIMA(0, 1, 1)",0.1,7,8.456862,25.512982,31.750949,39.826636
77,BNBUSD-N4Y,"ARIMA(1, 1, 0)",0.1,12,8.287234,25.289865,31.158957,38.906176
78,BNBUSD-N4Y,"ARIMA(1, 1, 0)",0.1,14,8.248705,25.498752,31.450442,39.063657
